# Custom Strategies
This notebook demonstrates how to define your own Control and Release Strategy

In [ ]:
from dataclasses import dataclass
from math import ceil
from supplychain_simulation import SupplyChain, Node, Sales, LeadTime, Edge, Simulator
from supplychain_simulation.node import Orders, Stock
from supplychain_simulation.pipeline import Pipeline, Receipt


## Control Strategy
The control strategy defines what orders to create for a Node in the supply-chain. 

In [ ]:
@dataclass
class RSQ:
    """RSQ implementation of the supply-chain control strategy

    Arguments:
        supply_chain(SupplyChain) SupplyChain instance to fetch the inventory levels from
    """

    supply_chain: SupplyChain

    def get_orders(self, node: Node, period: int) -> Orders:
        """Return the quantity of `node` to order"""
        data = node.data
        # Get the inventory level for the requested node
        inventory = self.supply_chain.inventory_assemblies_feasible(node)

        order_quantity = 0
        if (period % data["review_time"] == 0) and (inventory < data["reorder_level"]):
            order_quantity = (
                ceil((data["reorder_level"] - inventory) / data["order_quantity"])
                * data["order_quantity"]
            )
        orders = Orders()
        orders[node] = order_quantity
        return orders

## Release Strategy
The release strategy defines how much stock will be released from a Node and added to the pipeline

In [ ]:
class Fractional:
    """Fractional implementation of the supply-chain release strategy"""

    def get_releases(self, node: Node) -> Orders:
        """Build a set of Orders that should be released"""
        releases = Orders()
        orders = node.orders
        stock = node.stock[node]

        # determine the total amount ordered
        order_total = orders.sum()
        # if there are no orders
        if order_total == 0:
            # release nothing
            return releases

        # shortage can not be negative
        shortage = max(order_total - stock, 0)

        # create an order release where the shortage is divided relative to the
        # share ordered by each sku.
        for q_str, order in orders.items():
            releases[q_str] = ceil(order - shortage * (order / order_total))

        # In case the rounding caused more to be released than possible, reduce the largest
        # release by 1 until the total released equals the available stock.
        while releases.sum() > stock:
            max_order_release = max(releases, key=lambda x: releases[x])
            releases[max_order_release] -= 1

        return releases


## SupplyChain
The SupplyChain defines all the Nodes and reletions between those nodes.
Each Node contains information about the sales for that Node and the lead-time to apply for that Node

In [ ]:
# Build a supply-chain with 10 periods of sales
supply_chain = SupplyChain(
        nodes=[
            Node(
                "A",
                data=dict(
                    # Data needed for RSQ 
                    order_quantity=30,
                    reorder_level=25,
                    review_time=1,
                    safety_stock=1,
                ),
                
                # Simulation input data for this node
                sales=Sales({1: [1], 2: [4], 3: [10], 4: [12], 5: [10], 6: [11], 7: [8], 8: [10], 9: [10], 10: [10]}),
                lead_time=LeadTime(default=1),
                
                # Starting levels of stock/pipeline for this node
                stock=Stock({"A": 70, "C": 5}),
                pipeline=Pipeline([Receipt(sku_code="D", eta=0, quantity=35)]),
            ),
            Node(
                "B",
                data=dict(
                    order_quantity=25,
                    reorder_level=40,
                    review_time=1,
                ),
                backorders=5,
                sales=Sales({1: [15], 2: [15], 3: [15], 4: [14], 5: [2], 6: [17], 7: [18], 8: [10], 9: [12], 10: [15]}),
                lead_time=LeadTime(default=2),
                stock=Stock({"B": 30}),
                pipeline=Pipeline([Receipt(sku_code="D", eta=1, quantity=75)]),
            ),
            Node(
                "C",
                data=dict(
                    order_quantity=150,
                    reorder_level=20,
                    review_time=1,
                ),
                lead_time=LeadTime({1: 3, 2: 7}, default=3),
                stock=Stock({"C": 200}),
            ),
            Node(
                "D",
                data=dict(
                    order_quantity=200,
                    reorder_level=20,
                    review_time=2,
                ),
                lead_time=LeadTime(default=4),
                orders=Orders({"B": 15}),
                stock=Stock({"D": 40}),
            ),
        ],
        edges=[
            Edge(source="C", destination="A", number=2),
            Edge(source="D", destination="A", number=1),
            Edge(source="D", destination="B", number=3),
        ],
    )

In [ ]:
sim = Simulator(
    supply_chain,
    control_strategy=RSQ(supply_chain),
    release_strategy=Fractional(),
    
)

In [ ]:
sim.run(10)